# Sandbox: Ignore this

This notebook contains little experiments on doing things with python, jupyter, ipython et cetera
   It's not a central part of the project.
    
   Mostly ad hoc code snippets.  And not documented well

In [45]:
import os
import sys
import time
import datetime
import touch
import inspect
import datetime
# don't really need these below, just for testing
import numpy
from PIL import Image 
import plotnine as pn
import pandas as pd
print(sys.version_info)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


In [46]:
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.8.5 of Python
version re of re as 2.2.1 
No __version__ for touch as touch
version numpy of numpy as 1.19.3 
version Image of Image as 8.0.1 
version pn of plotnine as 0.7.1 
version pd of pandas as 1.1.3 
No __version__ for io as io
version np of numpy as 1.19.3 
version mpl of matplotlib as 3.3.3 
No __version__ for plt as plt
version fredapi of fredapi as 0.4.2 
version xlrd of xlrd as 1.2.0 
version selenium of selenium as 3.141.0 
version webdriver of webdriver as 3.14.1 
No __version__ for mdates as mdates
No __version__ for pytrends as pytrends
version pytz of pytz as 2020.1 
version nbformat of nbformat as 5.0.8 


In [47]:
# snippet to see the source code for a fuction
import inspect
import nbformat
fname = "nbformat.convert"
lines = inspect.getsource(eval(fname))
#print(lines)

In [48]:
# to get the names of notebook files in a directory
def get_project_notebooks():
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    proj_nbs = []
    nb_fnames = [f for f in os.listdir() if f.endswith(".ipynb")]
    for fn in nb_fnames:
        with open(fn) as fp:
            notebook = read(fp, as_version=NO_CONVERT)
        cells = notebook['cells']
        if not cells:
            continue
        src =cells[0]["source"]
        if not src:
            continue
        lines = src.split("\n")
        if lines[0].startswith("# "):
            proj_nbs.append(fn)
    return(proj_nbs)


# print the source code for a cell
def print_source(cell):
    print("type cell= {0}".format(type(cell)))
    src = cell["source"]
    lines = src.split("\n")
    print("{0} lines".format(len(lines)))
    for i,line in enumerate(lines):
        print("({0}){1}".format(i, line))

# run notebook code
def run_nb_code(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    for i, cell in enumerate(code_cells):
        src = cell['source']
        #print("{0}<{1}>".format(li, line))                
        try:
            exec(src, globals(), locals())
        except Exception as e:
            print("Error on cell {0}\n".format(i))
            print(src)
            print_source(cell)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            import traceback
            traceback.print_tb(exc_tb)
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            return((exc_type, exc_obj, exc_tb, fname))
    res += "end: " + str(datetime.datetime.now())
    return res
# run notebook code
def get_first_cell_line(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    if not cells:
        return ""
    lines = cells[0]["source"].split("\n")
    if not lines:
        return ""
    return lines[0]
def ordered_notebooks():
    nbs = get_project_notebooks()
    nbord = [None]*len(nbs)
    end_index = len(nbs)-1
    for nb in nbs:
        line0 = get_first_cell_line(nb)
        parts = line0.split(' ')
        index = -1
        if len(parts) >= 3 and parts[0]=='#' and parts[1]=='Part':
            try:
                index = int(parts[2]) - 1
            except Exception as e:
                pass
        if  index == -1:
            continue
        nbord[index] = nb
    nbord = [nb for nb in nbord if nb]
    return (nbord)
print(get_project_notebooks())
print(ordered_notebooks())

['combining_claims.ipynb', 'DataSources.ipynb', 'fastpages-howto.ipynb', 'foo.ipynb', 'fred.ipynb', 'google.ipynb', 'Intro_Overview.ipynb', 'setup.ipynb', 'Setup_old.ipynb', 'umich_cs.ipynb', 'weekly_claims.ipynb']
['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb']


In [49]:
from IPython.utils.capture import CapturedIO
from IPython.utils.capture import capture_output
nbs = ordered_notebooks()
print(nbs)
ofname = 'foo_out.txt'
with open(ofname, mode='w') as fp:
    fp.write("Datetime= {0}".format(str(datetime.datetime.now())))
outstr = ""
x = None
for nb in nbs:
    print("\nRunnning {0}".format(nb))
    with capture_output() as co:
        %run  $nb
        ofname = os.path.splitext(nb)[0] + ".out"
        with open(ofname, mode="w") as fp:
            fp.write(str(datetime.datetime.now()))
            fp.write(" stdout\n{0}\ns stderr{1}".format(co.stdout, co.stderr))
            for o in co.outputs:
                fp.write(str(o))

['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb']

Runnning Intro_Overview.ipynb

Runnning DataSources.ipynb

Runnning fred.ipynb

Runnning weekly_claims.ipynb

Runnning umich_cs.ipynb

Runnning google.ipynb

Runnning combining_claims.ipynb


In [50]:
# Touch them in order
import time
nbs = get_project_notebooks()
last  = len(nbs) - 1
nblist = [""]*len(nbs)
for nb in nbs:
    line = get_first_cell_line(nb)
    line_parts = line.split(" ")
    if len(line_parts) > 3 and line_parts[1] == "Part":
        try:
            index = int(line_parts[2]) - 1
        except:
            index = last
            last -= 1
    else:
        index = last
        last -= 1
    nblist[index] = nb
    #print(f"{nb}, index={index}, {line}")
print(nblist)
def add_str_to_file(fname, str="\n"):
    with open(fname, mode='a') as fp:
        fp.write(str)
    return
if False:
    for nb in nblist:
        try:
            print("{0} {1}".format(nb, datetime.datetime.now()))
            touch.touch(nb)
            add_str_to_file(nb, str="\n")
            time.sleep(61)
        except Exception as e:
            print(e)

['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb', 'Setup_old.ipynb', 'setup.ipynb', 'foo.ipynb', 'fastpages-howto.ipynb']


In [1]:
def print_vars(globs=True, locs=True, 
               filters={"exclude":("^_",)}):
    import re
    dicts = {"globs":globals().keys(),
            "locs":locals().keys()}
    for k,v in dicts.items():
        print("k={0}, eval(k)= {1}".format(k, eval(k)))
        if eval(k):
            y = [x for x in dicts[k]]
            if filters:
                if filters["exclude"]:
                    for filt in filters["exclude"]:
                        print("excluding: {0}".format(filt))
                        y = [x for x in y if not re.search(filt, x)]
            print("{0}= {1}".format(k,y))
        del y
print_vars()

k=globs, eval(k)= True
excluding: ^_
globs= ['In', 'Out', 'get_ipython', 'exit', 'quit', 'print_vars']
k=locs, eval(k)= True
excluding: ^_
locs= ['globs', 'locs', 'filters', 're', 'dicts', 'k', 'v', 'filt']
